# Allen Model

### Import and download all the dependencies

In [10]:
! pip install allennlp==2.1.0 allennlp-models==2.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 585 kB 4.1 MB/s 
     |████████████████████████████████| 407 kB 43.0 MB/s 
     |████████████████████████████████| 12.8 MB 35.7 MB/s 
     |████████████████████████████████| 1.9 MB 42.8 MB/s 
     |████████████████████████████████| 5.8 MB 24.1 MB/s 
     |████████████████████████████████| 132 kB 52.2 MB/s 
     |████████████████████████████████| 592 kB 63.5 MB/s 
     |████████████████████████████████| 776.8 MB 12 kB/s 
     |████████████████████████████████| 1.3 MB 32.3 MB/s 
     |████████████████████████████████| 125 kB 51.7 MB/s 
     |████████████████████████████████| 56 kB 2.9 MB/s 
     |████████████████████████████████| 53 kB 1.4 MB/s 
     |████████████████████████████████| 79 kB 7.8 MB/s 
     |████████████████████████████████| 9.1 MB 40.1 MB/s 
     |████████████████████████████████| 140 kB 55.0 MB/s 
     |███████████████████████████████

In [1]:
! pip install git+https://github.com/explosion/spacy-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/explosion/spacy-transformers to /tmp/pip-req-build-ktvoc49i
  Running command git clone -q https://github.com/explosion/spacy-transformers /tmp/pip-req-build-ktvoc49i
     |████████████████████████████████| 6.3 MB 4.0 MB/s 
     |████████████████████████████████| 1.1 MB 36.5 MB/s 
     |████████████████████████████████| 803 kB 62.5 MB/s 
  Created wheel for spacy-transformers: filename=spacy_transformers-1.1.8-py2.py3-none-any.whl size=54022 sha256=527e87da293de64ad746885c219add4dfd18a10ff03f99daf0c796dd2b22d529
  Stored in directory: /tmp/pip-ephem-wheel-cache-zh5u9rkn/wheels/41/c7/fa/69c1c02ea4105ea96b79ae6fd88420690edc71fff25e514f05
Successfully built spacy-transformers
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.0.17
  Attempting uninstall: spacy
    Found

### Divide the DataFrame to contains only `Context` and `Questions` columns.

In [16]:
import pandas as pd
df = pd.read_csv("context_ques.csv")
sub_df = df[["Context", "Question"]]
sub_df.head()

,Context,Question
0,The technological discipline of computer visio...,What are the types of computer vision tasks?
1,The technological discipline of computer visio...,What is the definition of computer vision?
2,Recent advances in deep learning has enabled r...,What is the role of the input to a device for ...
3,Recent advances in deep learning has enabled r...,What is the definition of computer vision?
4,"From the perspective of engineering, it seeks ...",What is the definition of computer vision?


### How to iterate over these two columns together?

In [17]:
for context, question in zip(sub_df.Context, sub_df.Question):
    print(context)
    print()
    print(question)
    break

The technological discipline of computer vision seeks to apply its theories and models to the construction of computer vision systems.Subdomains of computer vision include scene reconstruction, object detection, event detection, video tracking, object recognition, D pose estimation, learning, indexing, motion estimation, visual servoing, D scene modeling, and image restoration.Computer vision is an interdisciplinary field that deals with how computers can be made to gain highlevel understanding from digital images or videos.From the perspective of engineering, it seeks to understand and automate tasks that the human visual system can do.Computer vision tasks include methods for acquiring, processing, analyzing and understanding digital images, and extraction of highdimensional data from the real world in order to produce numerical or symbolic information, e.g.Some examples of typical computer vision tasks are presented below.Computer vision tasks include methods for acquiring, processi

### Now iterate over them and apply Allen on your DataFrame

In [18]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.rc

# This dictionary holds all the Allen outputs
pred_dict = []
  
for context, question in zip(sub_df.Context, sub_df.Question):
  predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/bidaf-elmo.2021-02-11.tar.gz")
  pred_dict.append(predictor.predict( passage= context, question= question))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:435: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")
/usr/local/lib/python3.7/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


### Build a list that contains all the question answers from Allen model which are the value of `best_span_str` key.

In [19]:
answers = []
for i in range(len(pred_dict)):
  answers.append(pred_dict[i]['best_span_str'])

## Check the output of the first question

In [20]:
question_1 = df.Question[0]
answer_1 = answers[1]
print("Question 1: {}".format(question_1))
print()
print("Answer: {}".format(answer_1))

Question 1: What are the types of computer vision tasks?

Answer: an interdisciplinary field that deals with how computers can be made to gain highlevel understanding from digital images or videos


# Final DataFrame

In [21]:
df.head()

,Unnamed: 0,Keyword,Context,Question
0,0,digital images,The technological discipline of computer visio...,What are the types of computer vision tasks?
1,1,computer vision,The technological discipline of computer visio...,What is the definition of computer vision?
2,2,processing,Recent advances in deep learning has enabled r...,What is the role of the input to a device for ...
3,3,highdimensional data,Recent advances in deep learning has enabled r...,What is the definition of computer vision?
4,4,object detection,"From the perspective of engineering, it seeks ...",What is the definition of computer vision?


In [22]:
df['Answers'] = answers

In [23]:
df.head()

,Unnamed: 0,Keyword,Context,Question,Answers
0,0,digital images,The technological discipline of computer visio...,What are the types of computer vision tasks?,"methods for acquiring, processing, analyzing a..."
1,1,computer vision,The technological discipline of computer visio...,What is the definition of computer vision?,an interdisciplinary field that deals with how...
2,2,processing,Recent advances in deep learning has enabled r...,What is the role of the input to a device for ...,vision sensor
3,3,highdimensional data,Recent advances in deep learning has enabled r...,What is the definition of computer vision?,extraction of information from image data to d...
4,4,object detection,"From the perspective of engineering, it seeks ...",What is the definition of computer vision?,"methods for acquiring, processing, analyzing a..."


In [25]:
df.drop("Unnamed: 0", axis=1, inplace=True)
df.head()

,Keyword,Context,Question,Answers
0,digital images,The technological discipline of computer visio...,What are the types of computer vision tasks?,"methods for acquiring, processing, analyzing a..."
1,computer vision,The technological discipline of computer visio...,What is the definition of computer vision?,an interdisciplinary field that deals with how...
2,processing,Recent advances in deep learning has enabled r...,What is the role of the input to a device for ...,vision sensor
3,highdimensional data,Recent advances in deep learning has enabled r...,What is the definition of computer vision?,extraction of information from image data to d...
4,object detection,"From the perspective of engineering, it seeks ...",What is the definition of computer vision?,"methods for acquiring, processing, analyzing a..."


In [26]:
df.to_csv("final_dataframe.csv")